**Глава 6 (Вежба 2) - Паралелизација во TensorFlow со TPUs**

Во вежбата се разгледува класификација на слики со конволуциска невронска мрежа со помош на FASHION MNIST податочното множество. Прикажано е како може моделот да биде трениран со паралелизација на еден или повеќе ТPUs. 

Примерот е преземен од https://www.tensorflow.org/guide/tpu и е соодветно прилагоден.





In [15]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [16]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True,try_gcs=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [17]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='') # специјална адресда за Colab
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.59.238.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.59.238.210:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [18]:
strategy = tf.distribute.TPUStrategy(resolver)


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [19]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

In [20]:
batch_size = 64

buffer_size=10000

train_dataset = mnist_train.map(scale).cache().shuffle(buffer_size).batch(batch_size)
eval_dataset = mnist_test.map(scale).batch(batch_size)

In [21]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 10s 11ms/step - loss: 0.2090 - sparse_categorical_accuracy: 0.9391
Epoch 2/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0681 - sparse_categorical_accuracy: 0.9800
Epoch 3/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0469 - sparse_categorical_accuracy: 0.9862
Epoch 4/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0344 - sparse_categorical_accuracy: 0.9896
Epoch 5/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9919
Epoch 6/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0200 - sparse_categorical_accuracy: 0.9936
Epoch 7/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0153 - sparse_categorical_accuracy: 0.9952
Epoch 8/10
938/938 [==============================] - 5s 5ms/step - loss: 0.0125 - sparse_categorical_accuracy: 0.9961
Epoch 9/10
938/938 [==========================

In [22]:
eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Евалуациска загуба: {}, Евалуациска точност: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 4s 27ms/step - loss: 0.0524 - sparse_categorical_accuracy: 0.9858
Евалуациска загуба: 0.0524231493473053, Евалуациска точност: 0.9857999682426453
